In [12]:
import os
from datetime import datetime
import pickle
import numpy as np
import tqdm
import json
from datetime import datetime
progress_bar = True
import time
import dynetx as dn
import openMindednessFunctions as omf

In [13]:
%cd datasets/in/

[WinError 3] Impossibile trovare il percorso specificato: 'datasets/in/'
C:\Users\valen\Github\openmindedness-gunminpol\datasets\in


# Estimating open mindedness distributions + other info

In [14]:
def createDataset(t0, tN):
    data = {}
    for dataset_name in ['minority', 'guncontrol', 'politics']:
        '''
        per ogni time step mi ritorna un dizionario nodo: {dizionario con delle info}
        '''
        data[dataset_name] = {}
        timestamps = omf.extract_timestamps(dataset_name)
        print(timestamps)
        t2node2opinions = omf.create_dictionaries(timestamps, dataset_name)
        for t in tqdm.tqdm(range(t0, tN)):
            data[dataset_name][t] = {}
            t1=t+1
            g = omf.createGraph(dataset_name, timestamps, t)
            for v in tqdm.tqdm(g.nodes()):
                if v in t2node2opinions[t1].keys():
                    opvt = t2node2opinions[t][v]
                    opvt1 = t2node2opinions[t1][v]
                    neighs = list(g.neighbors(v))
                    sortedNeighOps = omf.sortNeighOps(g, v, t, t2node2opinions, opvt)
                    eps, err, estOp = omf.estimation(opvt, opvt1, sortedNeighOps)
                    orientation = omf.politicalLeaning(opvt) 
                    nactiveint = len([op for op in sortedNeighOps if op <= eps])
                    data[dataset_name][t][v] = {'dataset':dataset_name, 'opt':opvt, 'opt1': opvt1, 'neighbors':neighs, 
                                                    'opChange':opvt1-opvt, 'homophily':0.0, 'orientation':orientation, 
                                                    'neighborsOps':sortedNeighOps, 'error':err, 'estOp':estOp, 
                                                    'openMindedness':eps, 'nactiveint':nactiveint}    
                    if t > t0:
                        try:
                            data[dataset_name][t][v]['orientation-1'] = data[t-1][v]['orientation']
                        except:
                            data[dataset_name][t][v]['orientation-1'] = None
    return data

In [15]:
data = createDataset(1, 5)
with open(f'openMindednessData.pickle', 'wb') as ofile:
    pickle.dump(data, ofile)

100%|██████████| 1040/1040 [00:00<00:00, 129808.24it/s]

100%|██████████| 1004/1004 [00:00<00:00, 166821.74it/s]

100%|██████████| 1170/1170 [00:00<00:00, 167046.86it/s]

100%|██████████| 833/833 [00:00<00:00, 118685.21it/s]

100%|██████████| 847/847 [00:00<00:00, 121293.85it/s]

100%|██████████| 1054/1054 [00:00<00:00, 176106.30it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{1: '2017-01-01_2017-07-01', 2: '2017-07-01_2018-01-01', 3: '2018-01-01_2018-07-01', 4: '2018-07-01_2019-01-01', 5: '2019-01-01_2019-07-01'}
{1: '2017-01-01_2017-07-01', 2: '2017-07-01_2018-01-01', 3: '2018-01-01_2018-07-01', 4: '2018-07-01_2019-01-01', 5: '2019-01-01_2019-07-01'}
{1: '2017-01-01_2017-07-01', 2: '2017-07-01_2018-01-01', 3: '2018-01-01_2018-07-01', 4: '2018-07-01_2019-01-01', 5: '2019-01-01_2019-07-01'}



100%|██████████| 917/917 [00:00<00:00, 227759.62it/s]

100%|██████████| 746/746 [00:00<00:00, 185485.26it/s]

100%|██████████| 825/825 [00:00<00:00, 165406.35it/s]

100%|██████████| 4/4 [00:00<00:00, 52.77it/s]


In [7]:
data

{'minority': {1: {4998: {'dataset': 'minority',
    'opt': 0.4994821595,
    'opt1': 0.4314617328125,
    'neighbors': [9822, 10273, 9628, 9837, 6988],
    'opChange': -0.06802042668750002,
    'homophily': 0.0,
    'orientation': 'Moderate',
    'neighborsOps': [0.5079718135,
     0.5244120212727272,
     0.5310620741645833,
     0.33016994452857146,
     0.027658792],
    'error': 0.005093866035477568,
    'estOp': 0.42636786677702243,
    'openMindedness': 0.16931221497142857,
    'nactiveint': 1},
   7058: {'dataset': 'minority',
    'opt': 0.8862351332727273,
    'opt1': 0.6290462926000001,
    'neighbors': [9577,
     1477,
     10339,
     6481,
     5104,
     7476,
     9787,
     10323,
     9122,
     9495,
     9985,
     9758],
    'opChange': -0.2571888406727272,
    'homophily': 0.0,
    'orientation': 'Republican',
    'neighborsOps': [0.7267302880000001,
     0.7267302880000001,
     0.6704663000000001,
     0.6623163046666667,
     0.6144391411238097,
     0.611081496